In [26]:
import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install matplotlib.pyplot
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install statsmodels
# !{sys.executable} -m pip install scikit-learn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
from scipy.interpolate import interp1d
from scipy.integrate import quad
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

In [27]:
df21 = pd.read_csv(r'C:\Users\rafae\OneDrive\Documentos\prog\coding\modelagem-estatistica\codes\df21', sep=',',encoding='iso-8859-1')
df22 = pd.read_csv(r'C:\Users\rafae\OneDrive\Documentos\prog\coding\modelagem-estatistica\codes\df22', sep=',',encoding='iso-8859-1')
df23 = pd.read_csv(r'C:\Users\rafae\OneDrive\Documentos\prog\coding\modelagem-estatistica\codes\df23', sep=',',encoding='iso-8859-1')

df = pd.concat([df21, df22, df23])

In [28]:
# Criando a coluna 'tema_relevante'
df['tema_relevante'] = df['NU_ANO'].apply(lambda x: 1 if x == 2022 else 0)

# Atualizando a coluna 'NU_ANO'
df['NU_ANO'] = df['NU_ANO'] - 2021

# Criando a coluna 'tema_relevante*raca_indigena'
df['tema_relevante*raca_indigena'] = df['tema_relevante'] * df['TP_COR_RACA_5']


In [29]:
X = df[['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESCOLA', 'NU_NOTA_CN',
       'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_COMP1', "NU_NOTA_COMP4",
       'Q022', 'Q025','TP_COR_RACA_2', 'TP_COR_RACA_3', 'TP_COR_RACA_4', 'TP_COR_RACA_5',
       'CAPITAL', 'Regiao_Centro-Oeste', 'Regiao_Nordeste', 'Regiao_Norte',
       'Regiao_Sul', 'faixa_per_capita_fam',
       'responsavel_maior_escolaridade', 'responsavel_maior_emprego',
       'tema_relevante','tema_relevante*raca_indigena']].copy()

Y = df['NU_NOTA_REDACAO'].copy()

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)

reg = LinearRegression()
# Treinando o modelo
model1 =  reg.fit(X_train, y_train)
# Prevendo os dados de saída com o modelo treinado 
y_pred = model1.predict(X_test)
# Aqui plotamos os coeficientes da nossa regressão
reg.coef_

array([ -2.06364803,  -2.90736423,   1.82418219,   0.05944753,
         0.09604187,   0.10398894,   0.07631636,   1.71060238,
         2.98388753,  -0.87811561,   4.32447414,   1.5551492 ,
        -0.16529932,  -6.05994087, -11.36548073,   0.13907993,
         1.65034521,  12.30886121,   5.05789348,  -5.54391597,
         0.45897488,   0.75248458,   0.47825514,   4.59985094,
         0.45642629])

In [30]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = sm.add_constant(X)

# Calculando VIF
vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif

,Variable,VIF
0,const,112.419213
1,TP_FAIXA_ETARIA,1.098143
2,TP_SEXO,1.102382
3,TP_ESCOLA,1.233581
4,NU_NOTA_CN,1.737362
5,NU_NOTA_CH,2.259742
6,NU_NOTA_LC,2.348330
7,NU_NOTA_MT,1.930579
8,NU_NOTA_COMP1,2.199832
9,NU_NOTA_COMP4,2.120571


In [31]:
print( mean_squared_error(y_test, y_pred) )
print( mean_absolute_error(y_test, y_pred) )

5511.196129993611
55.642390735075246


In [32]:
df_saida = pd.DataFrame()
df_saida['y'] = y_test
df_saida['y_pred'] = y_pred

# Visualizando os dados previstos vs dados reais.
df_saida.head(100)

,y,y_pred
2862,580.0,658.579998
6369,640.0,612.304899
5010,380.0,502.953516
8356,820.0,836.606675
4849,840.0,839.110049
...,...,...
3214,580.0,641.880721
1692,700.0,699.786819
4475,580.0,568.565650
2972,540.0,571.422640


In [33]:
# Importando Statsmodels para ter uma visão visão mais geral dos indicadores do modelo.
import statsmodels.api as sm

# Aqui estamos usando os mesmos dados anteriore de treino e teste. 
# O modelo resultante é exatamente o mesmo.
X_train=sm.add_constant(X_train)
m1 = sm.OLS( y_train, X_train)
m1 = m1.fit()
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        NU_NOTA_REDACAO   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     2973.
Date:                Mon, 17 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:17:40   Log-Likelihood:            -1.0900e+05
No. Observations:               19083   AIC:                         2.180e+05
Df Residuals:                   19057   BIC:                         2.183e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           -146.7851      5.613    -26.151      0.000    -157.787    -135.783
TP_FAIXA_ETARIA                   -2.0636      0.561     -3.681      0.000      -3.162      -0.965
TP_SEXO                           -2.9074      1.134     -2.564      0.010      -5.130      -0.685
TP_ESCOLA                          1.8242      2.632      0.693      0.488      -3.335       6.984
NU_NOTA_CN                         0.0594      0.010      5.796      0.000       0.039       0.080
NU_NOTA_CH                         0.0960      0.010      9.486      0.000       0.076       0.116
NU_NOTA_LC                         0.1040      0.011      9.254      0.000       0.082       0.126
NU_NOTA_MT                         0.0763      0.007     10.526      0.000       0.062       0.091
NU_NOTA_COMP1                      1.7106      0.031     55.506      0.000       1.650       1.771
NU_NOTA_COMP4                      2.9839      0.025    120.841      0.000       2.935       3.032
Q022                              -0.8781      0.522     -1.682      0.093      -1.901       0.145
Q025                               4.3245      1.502      2.878      0.004       1.380       7.269
TP_COR_RACA_2                      1.5551      2.430      0.640      0.522      -3.207       6.318
TP_COR_RACA_3                     -0.1653      1.736     -0.095      0.924      -3.569       3.238
TP_COR_RACA_4                     -6.0599      5.611     -1.080      0.280     -17.058       4.938
TP_COR_RACA_5                    -11.3655      1.772     -6.413      0.000     -14.839      -7.892
CAPITAL                            0.1391      1.385      0.100      0.920      -2.575       2.853
Regiao_Centro-Oeste                1.6503      2.076      0.795      0.427      -2.419       5.720
Regiao_Nordeste                   12.3089      1.488      8.271      0.000       9.392      15.226
Regiao_Norte                       5.0579      1.872      2.703      0.007       1.390       8.726
Regiao_Sul                        -5.5439      2.526     -2.195      0.028     -10.495      -0.593
faixa_per_capita_fam               0.4590      0.475      0.966      0.334      -0.472       1.390
responsavel_maior_escolaridade     0.7525      0.452      1.663      0.096      -0.134       1.639
responsavel_maior_emprego          0.4783      0.597      0.802      0.423      -0.691       1.648
tema_relevante                     4.5999      1.598      2.878      0.004       1.468       7.732
tema_relevante*raca_indigena       0.4564      2.255      0.202      0.840      -3.964       4.877
==============================================================================
Omnibus:                     2956.990   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (J